In [2]:
import pandas as pd

In [3]:
data = pd.read_csv('C:/project4/chat/id_history.csv', index_col=0, na_filter=False)

In [4]:
data.head()

,history
id,
rag123,


In [5]:
data.loc['rag123']

history    
Name: rag123, dtype: object

In [6]:
if 'rag123' in data.index:
    print('yes')

yes


In [7]:
data.loc['aab'] = {"history":""}
data

,history
id,
rag123,
aab,


In [8]:
def get_session_history(session_ids):
    print(f"[대화 세션ID]: {session_ids}")
    if session_ids not in data.index:  # 세션 ID가 data에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 data에 저장
        data.loc[session_ids] = {"history":""}
    return data.loc[session_ids].history  # 해당 세션 ID에 대한 세션 기록 반환

In [9]:
get_session_history('tjr')

[대화 세션ID]: tjr


''

In [10]:
data

,history
id,
rag123,
aab,
tjr,


In [11]:
from langchain.prompts import HumanMessagePromptTemplate, ChatPromptTemplate
from langchain.schema import SystemMessage
from langchain.chat_models import ChatOpenAI
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv

load_dotenv()



# page_content만 저장
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# 모델
chatgpt = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature = 0.3
)

# 챗 메세지
chat_messages = [
    SystemMessage(content='당신은 해양 관련 지식을 가지고 있는 사람입니다. 질문에 대해 100자 내외로 말해주고, 해양과 관련된 이야기가 아니면 정중하게 거절해주세요.'),
    HumanMessagePromptTemplate.from_template('{history}'),
    HumanMessagePromptTemplate.from_template('Context: {context}\nQuestion: {ques}')
]

# 챗 프롬프트
chat_prompt = ChatPromptTemplate.from_messages(chat_messages)

vectorstore = Chroma(embedding_function=OpenAIEmbeddings(), persist_directory='C:/project4/chat/testDB')
retriever = vectorstore.as_retriever(search_kwargs={"k":3})


while True:
    id = input('id')
    history = get_session_history(id)
    try:
        q = input('ques')
        
        context = format_docs(retriever.invoke(history+q))
        
        result = chat_prompt.invoke({
            'history': history,
            'context': context,
            'ques': q
        })

        ans = chatgpt.invoke(result).content
        print(ans)
        
        data.loc[id].history += f'Human: {q}\nAI: {ans}\n'

    except:
        print(data[id])
        break

[대화 세션ID]: ddd
안녕하세요! 해양 관련 질문이 있으시면 언제든지 물어보세요. 도움이 필요하시면 기꺼이 도와드리겠습니다.
[대화 세션ID]: ddd
해양쓰레기는 주로 두 가지 출처에서 발생합니다. 첫째, 육상 기인 해양쓰레기로, 육지에서 발생한 플라스틱, 비닐봉지, 음료수 병 등이 바다로 유입됩니다. 둘째, 해상 기인 쓰레기로, 해상에서 발생한 어구, 부표 등이 바다에 버려지거나 떠내려옵니다.


KeyboardInterrupt: Interrupted by user

In [12]:
data

,history
id,
rag123,
aab,
tjr,
ddd,Human: 안녕\nAI: 안녕하세요! 해양 관련 질문이 있으시면 언제든지 물어보세...


In [13]:
data.to_csv('C:/project4/chat/id_history.csv')